<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo3/M5U3_Introducci%C3%B3n_Redes_Neuronales_Convolucionales_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1kl2OFnF2FADAAKjgZUFPo8dsBQUSIJM7" alt = "Encabezado MLDS" width = "100%">  </img>


#**Introducción a las Redes Neuronales Convolucionales (CNN)**
----

Como vimos en la unidad anterior, **_Keras_** ofrece una gran variedad de capas o *layers* para la construcción de modelos. En esta unidad, nos concentraremos en estudiar **las redes neuronales convolucionales**, las cuales son uno de los modelos más usados para el análisis de imagenes. Inicialmente, se mostrarán dos de los componentes principales de estas redes: **la convolución y el _pooling_**. Finalmente se presentará un ejemplo práctico de este tipo de red.

Veremos:

- Capas convolucionales
- Capas de *Pooling*
- Redes Neuronales Convolucionales
- *Data augmentation*
- Entrenamiento y visualización



Primero importatemos los paquetes necesarios :

In [ ]:
# Seleccionamos la versión más reciente de Tensorflow 2.0
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os, random
%matplotlib inline
plt.style.use("ggplot")
# Seleccionamos una semilla para los RNG
tf.random.set_seed(0)
np.random.seed(0)

# **1. Introducción**
----

Dos de las tareas más populares en el análisis de imágenes automático son la **identificación y clasificación de objetos dentro de una imagen**.

Anteriormente, hemos estudiado las redes neuronales multicapa, las cuales pueden usarse en tareas tanto de regresión como de clasificación. En general, se tratan de modelos basados en transformaciones no líneales (gracias a la función de activación) de sumas ponderadas de un vector de entrada. Sin embargo, cuando trabajamos con **imágenes**, no es muy apropiado utilizar directamente un perceptrón multicapa debido a las siguientes razones :

  1) El perceptrón multicapa encuentra **patrones fijos** en las variables, no obstante, no es capaz de capturar propiedades comunes de las imágenes como la invarianza a escala, la rotación y la traslación.

  2) Es un modelo que **conecta densamente** cada una de las variables de entrada con la capa siguiente. Esto es un inconveniente cuando los datos que tenemos tienen muchas característica como las imágenes, las cuales se caracterizan por estar compuestas por muchos **pixeles**.

<center><img src="https://drive.google.com/uc?export=view&id=1Wn7EQ8Ex0mgca0B07ReW5uyEAf1N4Q3H" width="40%" /></center>

  - Por ejemplo, si tenemos imágenes a color (RGB) de tamaño $256\times256$, el vector de entrada de la red sería de tamaño $196.608$ ($256\times256\times3$). Así mismo, si quisiéramos utilizar una capa intermedia de tamaño 8 (lo cual es un valor muy pequeño para representar toda la información en una imagen) necesitaríamos una matriz de pesos de tamaño $1'572.864$. Es decir, el número de pesos del modelo puede crecer muy fácilmente.

Una alternativa a lo expuesto anteriormente es determinar **un conjunto de características** que capturen de forma apropiada la información en las imágenes y que sean de una dimensión apropiada para poder utilizar un perceptrón multicapa.

- A este proceso se le conoce como **feature engineering** y es un campo que ha sido muy estudiado en áreas como el procesamiento de imágenes y visión por computador.

No obstante, determinar un conjunto de características apropiado para una tarea en específico puede llegar a ser dispendioso y difícil. Por ello, surge un gran interés en métodos que puedan aprender a extraer características de forma automática, lo cual es conocido como **feature learning**.


## 1.1 Redes neuronales convolucionales

Las **redes neuronales convolucionales** o *Convolutional Neural Networks* (CNN), están inspiradas en los procesos biológicos que tienen lugar en la corteza cerebral, donde las neuronas individuales responden a estímulos de un área restringida del campo visual. Las CNNs son capaces de aprender características visuales de bajo nivel que después son agrupadas en patrones de más alto nivel llevando a cabo un proceso de *feauture learning*.



<center><img src="https://drive.google.com/uc?export=view&id=1NYS3i5LhYAsrlMyzPf9aJe5QuVtXqYy6" width="90%" /></center>

A diferencia del perceptrón multicapa, en una CNN **los pesos no se asignan a un píxel en específico** sino que son compartidos por varios pixeles por medio de una operación conocida como **convolución** (la cual no se ve afectada por efectos de traslación), además, incluyen etapas de submuestreo que permiten analizar una imagen a varias escalas. Como resultado, las CNNs aprenden a responder a diferentes características en una imagen (bordes, formas, entre otros), tal y como funcionan los bancos de filtros (los cuales requieren ser definidos de forma manual) comúnmente usados en los algoritmos tradicionales.
- La capacidad de aprender dichos filtros supone una **ventaja única de las CNNs**, que elimina el esfuerzo manual requerido en el diseño de características.  



# **2. Convolución**
----

La **convolución** es conocida por que es ampliamente usada en distintas aplicaciones como :
- Ecuaciones diferenciales
- Procesamiento de señales
- Procesamiento de imágenes
- Visión por computador.

Se trata de **una operación matemática** resultante de la suma ponderada entre una imagen $\mathbf{I}$ y un filtro móvil $\mathbf{H}$. Este concepto se puede entender más fácilmente de forma gráfica, tal y como se muestra a continuación:

<center><img src="https://drive.google.com/uc?export=view&id=1QIRMoaS4xVBjCRo9XqwK42AGyGeeIqo2" width="50%" /></center>

En este caso, un filtro de dimensión $3\times 3$ recorre la imagen (convolución 2D) dando pasos de un pixel y estará conformado por los siguientes elementos:

$$ \mathbf{H}=\left(\begin{array}{cc}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0 \\  
\end{array}\right)$$

Los elementos de dicho filtro son multiplicados **elemento a elemento** por una sección de la imágen. Posteriormente, los resultados obtenidos son sumados de forma que 9 pixeles dentro de la imagen original pasan a ser representados por un único pixel.

- Adicionalmente, la convolución se puede aplicar en imágenes a color (RGB), repitiendo la operación en cada canal como se muestra a continuación :
<center><img src="https://drive.google.com/uc?export=view&id=1QNyZEP-Jvc7WfVMsrUoFNmEzyfRXJTho" width="60%" /></center>


<center><img src="https://drive.google.com/uc?export=view&id=1dUzEHGNDI6w7-7je_E4NwHzDDaUN66Zb" width="40%" /></center>

La convolución tiene varias propiedades que la hacen bastante útil, por ejemplo :
- Permite **aproximar diferencias finitas** de cualquier orden (gradiente, laplaciano, entre otros).
- Permite **suavizar imagenes** (para eliminar ruido) e incluso puede ser usada para resaltar detalles o patrones.

Tradicionalmente, los filtros se diseñaban manualmente para cada tarea en específico. Veamos algunos ejemplos de filtros y el resultado de una convolución utilizando _TensorFlow_ :

In [ ]:
# Utilizaremos una imagen de ejemplo tomada de: Michael Plotke - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=24301122
!wget -O animal.png https://upload.wikimedia.org/wikipedia/commons/5/50/Vd-Orig.png?20130129005336

In [ ]:
# Cargamos la imagen en formato RGB
im = np.array(tf.keras.preprocessing.image.load_img("animal.png"))
print(f"Dimensiones de la imagen a color: {im.shape}")
# Convertimos la imagen a grises
im_gray = tf.constant(np.mean(im,axis=2).astype(np.float32))
print(f"Dimensiones de la imagen en escala de grises: {im_gray.shape}\n\n\n")
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.imshow(im)
plt.axis("off"); plt.title("Imágen de ejemplo")
plt.subplot(122)
plt.imshow(im_gray,cmap="gray")
plt.axis("off"); plt.title("Imágen en grises")

Ahora, veamos el efecto de aplicar una convolución. Para el ejemplo, usaremos filtros **Sobel**, usados para detección de bordes.

In [ ]:
# Definimos un filtro sobel horizontal
# -1  0  1
# -2  0  2
# -1  0  1
H1 = tf.constant([[-1, 0, 1],
                  [-2, 0, 2],
                  [-1, 0, 1]], tf.float32)
# Ajustamos las dimensiones del kernel para poder usar la función de Tensorflow
H1 = tf.reshape(H1, [3, 3, 1, 1])

# Definimos un filtro sobel vertical
#  1  2  1
#  0  0  0
# -1 -2 -1
H2 = tf.transpose(tf.constant([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]], tf.float32))
# Ajustamos las dimensiones del kernel para poder usar la función de Tensorflow
H2 = tf.reshape(H2, [3, 3, 1, 1])

# Definimos un filtro de derivada
#  0  1  0
#  1 -4  1
#  0  1  0
H3 = tf.transpose(tf.constant([[0,  1,  0],
                               [1, -4,  1],
                               [0,  1,  0]], tf.float32))
# Ajustamos las dimensiones del kernel para poder usar la función de Tensorflow
H3 = tf.reshape(H3, [3, 3, 1, 1])

# Definimos un filtro box blur
# 1/9 1/9 1/9
# 1/9 1/9 1/9
# 1/9 1/9 1/9

h4 = np.zeros((3, 3, 3, 3))
# Asignamos el filtro para cada canal
for i in range(3):
    h4[:, :, i, i] = np.ones((3, 3))/9.
# Construimos el tensor
H4 = tf.constant(h4, tf.float32)

# Ajustamos las dimensiones de la imagen
image_resized = tf.expand_dims(tf.expand_dims(im_gray, 0), 3)
print(f"Dimensiones de la imagen ajustadas: {image_resized.shape}\n\n")

Aplicamos los primeros tres filtros sobre la imagen en grises y mostramos los resultados. Usaremos la función `tf.nn.conv2d` que nos permite aplicar convoluciones definiendo kernels o filtros particulares. `tf.nn.conv2d` recible los siguientes argumentos:

*  `input`: Un tensor de rango al menos 4. Por lo general la primera dimensión se asocia al indice en el batch de imágenes, y las últimas 3 a las dimensiones propias de las imágenes.
*  `filters`: Un tensor 4-D de forma `[altura_filtro, anchura_filtro, canales_entrada, canales_salida]`. Es el filtro convolucional.
*  `strides`: Un `int` o lista de `int`'s de longitud 1, 2 ó 4. El tamaño de paso de la ventana deslizante para cada dimensión de entrada. Si se da un único valor, se replica en todas las dimensiones.
*  `padding`: Puede ser la cadena `"SAME"` o `"VALID"`, que indica el tipo de algoritmo de relleno que se va a utilizar. `"VALID"` indica que no se utilice relleno. Esto hace que el tamaño de salida sea normalmente menor que el tamaño de entrada, incluso cuando el `stride` es 1. Con el relleno `"SAME"`, el relleno se aplica a cada dimensión espacial. Cuando el `stride` es 1, la entrada se rellena de forma que el tamaño de salida sea el mismo que el de entrada.



In [ ]:
I1 = tf.nn.conv2d(image_resized, filters=H1,
                  strides=[1, 1], padding='SAME')
I2 = tf.nn.conv2d(image_resized, filters=H2,
                  strides=[1, 1], padding='SAME')
I3 = tf.nn.conv2d(image_resized, filters=H3,
                  strides=[1, 1], padding='SAME')

# Mostramos los resultados

ims=[I1[0,:,:,0],I2[0,:,:,0],I3[0,:,:,0]]
names=["Sobel x", "Sobel y", "Derivada"]
fig,ax=plt.subplots(1,3,figsize=(10,10))
for i in range(3):
    ax[i].imshow(ims[i], cmap="gray")
    ax[i].set_title(names[i]); ax[i].axis("off")

Ahora aplicamos el último filtro sobre la imagen a color:

In [ ]:
image_resized = tf.expand_dims(tf.constant(im, tf.float32), 0)
I4 = tf.nn.conv2d(image_resized, filters=H4,
                  strides=[1, 1], padding='SAME')

# Mostramos los resultados

ims = tf.cast(I4[0, :, :, :],tf.uint8)
name = "Box Blur"
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.imshow(ims, cmap="gray")
ax.set_title(name); ax.axis("off")

_Keras_ provee una capa especial de convolución `tf.keras.layers.Conv2D` para ser utilizada en conjunto con las redes neuronales. Se trata de un bloque funcional que contiene unos pesos (varios filtros) y permite realizar la operación de convolución. A diferencia de `tf.nn.conv2d`, con `tf.keras.layers.Conv2D` no definimos explicitamente los filtros; definimos cuántos filtros vamos a usar, y se aprenden luego durante el entrenamiento. `tf.keras.layers.Conv2D` tiene los siguientes parámetros:

<center><img src="https://drive.google.com/uc?export=view&id=1kikAVAeIowl-uvT6xIDP5mGotFFocFvt" width="80%" /></center>

* ```filters```: El número de filtros. Una capa convolucional se compone de varias operaciones de convolución, es decir, contiene varios filtros.
* ```kernel_size```: Define el tamaño de los filtros que se utilizarán en las convoluciones, por ejemplo, la tupla ```(3,3)``` define una matríz de tamaño $3\times 3$.
* ```strides```: Se trata del tamaño del salto en la convolución. Es decir, permite controlar el número de pixeles que se desplaza el filtro en cada iteración de la convolución. Por ejemplo, la tupla ```(2,1)``` define que el filtro se mueve de a dos pixeles en el primer eje y de a un pixel en el segundo.
* ```padding```: Define cómo se manejan los bordes de la imagen resultante. Por un lado, una convolución de tamaño igual (```"same"```) mantendrá las dimensiones de la imagen resultante del mismo tamaño de la imagen original al agregar ceros alrededor de los límites de la imagen de entrada cuando sea necesario. Por otro lado, una convolución válida (```"valid"```) sólo realiza la convolución en los pixeles donde sea posible, es decir, la convolución no se aplica en los límites y por ende el tamaño de la imagen resultante es menor.
* ```activation```: Al igual que en las redes multicapa, se puede utilizar una función de activación para agregar no-linealidad a las representaciones.

_Keras_ inicializa los filtros de una capa convolucional de forma aleatoria para posteriormente aprenderlos, veamos un ejemplo de esto:


In [ ]:
# Definimos una capa de entrada
inp = tf.keras.layers.Input(shape=(100, 100, 3))
# Definimos una capa convolucional de ejemplo conectada a la entrada
conv = tf.keras.layers.Conv2D(filters=4,
                              kernel_size=(5,5),
                              strides=(1,1),
                              padding="same",
                              activation=tf.nn.sigmoid)(inp)
# Definimos un modelo
model = tf.keras.models.Model(inputs=[inp], outputs=[conv])
model.summary()

Veamos los filtros de la capa convolucional :

In [ ]:
model.layers[1].weights

In [ ]:
model.layers[1].weights[0].numpy()[:, :, 0, 0]

Podemos ver que los filtros tienen tamaño ```(5, 5, 3, 4)```. Los primeros dos números corresponden al tamaño del filtro, el tercero corresponde al número de canales de la imágen de entrada y el cuarto al número de filtros en la capa convolucional.
- Veamos un ejemplo de las convoluciones iniciales (pesos aleatorios).

In [ ]:
res = model(tf.cast(tf.expand_dims(im, 0), tf.float32))
print(f"Dimensiones del resultado de la convolución: {res.shape}\n\n\n")


plt.figure(figsize=(10,10))
for i in range(4):
    plt.subplot(221+i)
    plt.imshow(res[0,:,:,i], "gray")
    plt.title(f" Filtro {i+1}")
    plt.axis("off")

# **3. Pooling**
----

Otro componente fundamental en las redes convolucionales es el **_pooling_** o submuestreo, el cual se utiliza para **reducir la resolución de una imagen** con el fin de evitar que la red aprenda patrones muy detallados (sobreajuste). Además, permite que la red aprenda patrones en distintas escalas y disminuya el número total de parámetros en un modelo.

Generalmente, la operación de *pooling* se realiza utilizando una función que reduce el tamaño de la imagen original siguiendo una regla en específico, algunos ejemplos de esto son :

* ```tf.keras.layers.AveragePooling2D```: Reduce la dimensión de una imagen al obtener el promedio en determinados vecindarios.
* ```tf.keras.layers.MaxPooling2D```: Reduce la dimensión de una imagen al representar cada vecindario por el pixel de mayor intensidad.

Un ejemplo de diferentes funciones de pooling se ilustra en la siguiente figura :

<center><img src="https://drive.google.com/uc?export=view&id=1iYFdNoS1iqo_X2kpm9o6DsgqUC5vkiur" width="60%" /></center>

Las funciones de Pooling reciben parámetros similares a las capas de convolución, que controlan factores como el tamaño de la ventana y el tamaño del paso. Esto a su vez, define el tamaño de la salida de la capa. Veamos en detalle:

*   `pool_size`: es el tamaño del vecindario o de la ventana.
*   `strides`: el tamaño de paso en cada dimensión que debe moverse la ventana.
*   `padding`: define el manejo de los bordes de la entrada a la capa. Analogo a lo que sucede en las capas convolucionales, un padding válido o `"valid"` indica que no se hace relleno sobre los bordes del tensor, resultando en una reducción de tamaño en la salida, y  `"same"` indica que el relleno se ajusta para que la salida conserve las dimensines de la entrada.

En el caso del ejemplo anterior, se utilizan vecindarios (`pool_size`) de tamaño `(2,2)`, se utilizan pasos (`strides`) de tamaño `(2,2)` y se utiliza un *padding* válido.

- Otros tipos de pooling pueden ser consultados en la documentación y se encuentran en ```tf.keras.layers```.

Ahora veamos un ejemplo de pooling en _Tensorflow_ :

In [ ]:
# Definimos una capa de entrada
inp = tf.keras.layers.Input(shape=(100, 100, 3))
# Definimos una capa de average pooling
pool = tf.keras.layers.AveragePooling2D(pool_size=(5,5),
                                        strides=(5,5),
                                        padding="valid")(inp)
# Definimos un modelo
model = tf.keras.models.Model(inputs=[inp], outputs=[pool])
model.summary()

En el ejemplo anterior, el tensor de entrada tiene tamaño `(100, 100, 3)`. Al apliacar un Pooling con `pool_size=(5,5)` y `strides=(5,5)` quiere decir que la ventana de tamaño `(5,5)` se mueve cada vez 5 posiciones en ambas direcciones. Por tanto, caben 20 filtros a lo alto y ancho del tensor de entrada, lo que arroja una salida de tamaño `(20,20,3)`.

En el siguiente ejemplo, cambia el `pool_size` a `(10,10)`, pero se mantiene el `strides`, entonces ya no caben 20 filtros sino 19 a lo largo y ancho del tensor de entrada:

In [ ]:
# Definimos una capa de entrada
inp = tf.keras.layers.Input(shape=(100, 100, 3))
# Definimos una capa de max pooling
pool = tf.keras.layers.MaxPooling2D(pool_size=(10,10), strides=(5,5),
                                    padding="valid")(inp)
# Definimos un modelo
model2 = tf.keras.models.Model(inputs=[inp], outputs=[pool])
model2.summary()

In [ ]:
# Obtenemos el resultado del primer modelo
res_avg = model(tf.cast(tf.expand_dims(im, 0), tf.float32))
# Obtenemos el resultado del segundo modelo
res_max = model2(tf.cast(tf.expand_dims(im, 0), tf.float32))

plt.figure(figsize=(12,4))
plt.subplot(131)
plt.imshow(im); plt.title("Imagen Original"); plt.axis("off")
plt.subplot(132)
plt.imshow(tf.cast(res_avg[0], tf.uint8)); plt.title("Average Pooling"); plt.axis("off")
plt.subplot(133)
plt.imshow(tf.cast(res_max[0], tf.uint8)); plt.title("Maximum Pooling"); plt.axis("off")

# **4. Redes neuronales convolucionales**
----

Una arquitectura de red neuronal convolucional se compone principalmente de **capas convoluciones y _poolings_** realizados de forma secuencial. Con esto, la red puede obtener varias representaciones intermedias (*feature maps*) hasta llegar a un nivel de abstracción suficiente para realizar una predicción apropiada (clasificación o regresión). Más específicamente, este tipo de arquitecturas incluyen un predictor al final (generalmente una red neuronal multicapa) que utiliza una representación más simple de una imagen (análogo al enfoque clásico donde se extraían características de una imagen para entrenar un clasificador).

<center><img src="https://drive.google.com/uc?export=view&id=12xLj0Tfc7Df-n0HIenyX_7QGPNv2q81r" width="90%" /></center>


Veamos cómo construír una red neuronal convolucional en _TensorFlow_:

Definimos un modelo en keras

In [ ]:
conv_net = tf.keras.models.Sequential()

Definimos una capa de entrada, especificamos que el tamaño de nuestras imágenes será de 150x150 y que tendrá tres canales.

In [ ]:
conv_net.add(tf.keras.layers.Input(shape=(150, 150, 3)))

Agregamos bloques de convolución seguidos de max pooling

- Primer bloque

In [ ]:
conv_net.add(tf.keras.layers.Conv2D(filters=36,
                                    kernel_size=3,
                                    activation='relu'))

conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

- Segundo bloque

In [ ]:
conv_net.add(tf.keras.layers.Conv2D(filters=36,
                                    kernel_size=3,
                                    activation='relu'))

conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

- Tercer bloque

In [ ]:
conv_net.add(tf.keras.layers.Conv2D(filters=36,
                                    kernel_size=3,
                                    activation='relu'))

conv_net.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

Agregamos una capa `flatten`, que transforma cualquier arreglo multidimensional en un vector unidimensional.

In [ ]:
conv_net.add(tf.keras.layers.Flatten())

Y finalmente agregamos un clasificador, en este caso una red neuronal multicapa:

In [ ]:
# Capa densa intermedia
conv_net.add(tf.keras.layers.Dense(units=512,
                                   activation='relu'))
# Capa de salida
conv_net.add(tf.keras.layers.Dense(units=1,
                                   activation='sigmoid'))

Visualicemos:

In [ ]:
conv_net.summary()
tf.keras.utils.plot_model(conv_net,show_shapes=True)

# **5. Aumentación de datos (_Data Augmentation_)**
----

<center><img src="https://drive.google.com/uc?export=view&id=1mUGU94zRIuMMVhfhdsHwsAOtqOmI5IC1" width="60%" /></center>

Una de las características de las redes neuronales convolucionales es que **necesitan una gran cantidad de datos para aprender**. No obstante, existen alternativas para hacer que este tipo de modelos aprendan en conjuntos de datos pequeños. La idea es extender el conjunto de entrenamiento por medio de la generación de datos o *data augmentation*, en el caso específico de imágenes, consiste en generar nuevas imágenes transformadas (cambios en traslación, rotación, intensidad, entre otros) a partir del conjunto de imágenes original.

_Keras_ provee herramientas que nos permiten realizar *data augmentation* y que pueden ser utilizadas en conjunto con los modelos. Para ilustrar esto, utilizaremos un dataset con imágenes de perros y gatos. Usaremos el siguiente código para descargar el dataset:

In [ ]:
# limpiamos el dataset
!rm -rf /tmp/cats_and_dogs_*
# descargamos el dataset
!wget --no-check-certificate \
     https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
     -O /tmp/cats_and_dogs_filtered.zip
# descomprimimos
!unzip /tmp/cats_and_dogs_filtered.zip -d /tmp/

Los contenidos del zip se extraen al directorio base ```/tmp/cats_and_dogs_filtered```, el cual contiene subdirectorios para los conjuntos de datos de entrenamiento ```/tmp/cats_and_dogs_filtered/train/```y validación ```/tmp/cats_and_dogs_filtered/validation/```. De igual forma, cada uno de estos subdirectorios contiene carpetas con imágenes por cada categoría, es decir, una para perros y una para gatos.

Comencemos de definiendo las rutas del dataset:

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directorio con las imágenes de gatos para entrenamiento
train_cats_dir = os.path.join(train_dir, 'cats')

# Directorio con las imágenes de perros para entrenamiento
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directorio con las imágenes de gatos para validación
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directorio con las imágenes de perros para validación
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Veamos un ejemplo de los nombres algunas imágenes de cada categoría:

In [ ]:
train_cat_fnames = os.listdir(train_cats_dir)
print(train_cat_fnames[:10])

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print(train_dog_fnames[:10])

El dataset está constituído de la siguiente forma :

In [ ]:
print('Imágenes de gatos en entrenamiento:', len(os.listdir(train_cats_dir)))
print('Imágenes de perros en entrenamiento:', len(os.listdir(train_dogs_dir)))
print('Imágenes de gatos en validación:', len(os.listdir(validation_cats_dir)))
print('Imágenes de perros en validación:', len(os.listdir(validation_dogs_dir)))

Ahora, veamos algunos ejemplos de las imágenes originales :

In [ ]:
fig, ax = plt.subplots(2, 8, figsize=(15,10))

# seleccionamos las primeras 8 imágenes de cada categoría
next_cat_pix = [os.path.join(train_cats_dir, fname)
                for fname in train_cat_fnames[:8]]
next_dog_pix = [os.path.join(train_dogs_dir, fname)
                for fname in train_dog_fnames[:8]]

# mostramos las imágenes
for i, img_path in enumerate(next_cat_pix+next_dog_pix):
    img = tf.keras.preprocessing.image.load_img(img_path)
    ax[i//8,i%8].imshow(img)
    ax[i//8,i%8].axis("off")
plt.tight_layout()

Para realizar *data augmentation* utilizamos el objeto ```tf.keras.preprocessing.image.ImageDataGenerator```, el cual requiere los siguientes argumentos:

* ```rescale```: Define una normalización de intensidad utilizando un factor dado.
* ```rotation_range```: Define un rango (a nivel de ángulos) en el que se generarán rotaciones aleatorias de las imágenes.
* ```width_shift_range```: Define un rango en el que se generarán traslaciones horizontales de las imágenes.
* ```height_shift_range```: Define un rango en el que se generarán traslaciones verticales de las imágenes.
* ```shear_range```: Define un rango en el que se generará el efecto de cizallamiento o *shear* .
* ```zoom_range```: Define un rango en el que se generará el efecto aumento o *zoom*.
* ```horizontal_flip```: Se especifica si las imágenes generadas se pueden invertir horizontalmente.
* ```vertical_flip```: Se especifica si las imágenes generadas se pueden invertir verticalmente.
* ```fill_mode```: en algunas transformaciones aparecen espacios que no estaban contenidos en la imagen original, este parámetro permite especificar si estos espacios se llenarán con el pixel más cercano ```"nearest"``` o con un valor constante ```"constant"```.

Veamos un ejemplo de esto:

In [ ]:
# Definimos las transformaciones para el conjunto de train
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                rotation_range=40,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True,
                                                                fill_mode='constant')

# Definimos las transformaciones para el conjunto de test
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

Note que para el conjunto de validación no hacemos transformaciones más allá de `rescale`. Esto es porque el conjunto de validación lo usamos para medir el desempeño del modelo, y esto debe hacerse siempre con los datos inalterados, tal cual como se espera que lleguen en una aplicación real. De igual manera, no se hace augmentation en el conjunto de prueba final.

Ahora, obtenemos generadores de _keras_ que permitirán el entrenamiento por batch. `ImageDataGenerator` tiene una función llamada `flow_from_directory` que se encarga de pasarle al modelo grupos de muestras o _batches_. `flow_from_directory` requiere los siguientes argumentos:

*   `directory`: la ruta al directorio donde se encuentran los datos.
*   `target_size`: el tamaño (`altura`,`ancho`) al que deben reformarce las imágenes para entrar al modelo.
*   `batch_size`: el número de muestras por batch.
*   `class_mode`: puede ser `categorical`, `binary`, `sparse`, `input`, o `None`. Determina el tipo de etiqueta que se usará en el modelo. `categorical` hará que las etiquetas tengan una codificación _one_hot_. `binary` o `sparse` hará que las etiquetas sean números enteros. `None` no alimentará al modelo con etiquetas.

Con esto definimos entonces dos _generators_, uno para entrenamiento y otro para validación:



In [ ]:
# Especificamos el tamaño del batch, número de imagenes que genera en cada iteración
batch_size = 128

# Obtenemos un generador que realiza las transformaciones y carga
# las imágenes de entrenamiento
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# Obtenemos un generador que realiza las transformaciones y carga
# las imágenes de validación
validation_generator = val_datagen.flow_from_directory(directory=validation_dir,
                                                       target_size=(150, 150),
                                                       batch_size=batch_size,
                                                       class_mode='binary')

Usando la función `next()`, que recibe como argumento un **generador**, veamos un ejemplo de las imágenes generadas :

In [ ]:
# Extraemos un batch
ims, lab = next(train_generator)

plt.figure(figsize=(10, 10))
# Mostramos 9 ejemplos
for i in range(9):
    plt.subplot(331+i)
    plt.imshow(ims[i]); plt.axis("off"); plt.title(f"Label: {lab[i]}")

# **6. Entrenamiento y visualización**
----

Para entrenar la red neuronal convolucional con los generadores que realizan el **_data augmentation_** utilizaremos la función usual ```fit```, que puede recibir un **generador** en lugar de arreglos multidimensionales explícitos. En este caso, al igual que con los datasets de _TensorFlow_, debemos especificar el número de batches en cada época.

Primero, compilemos el modelo:

In [ ]:
conv_net.compile(loss="binary_crossentropy",
                 optimizer=tf.optimizers.Adam(learning_rate=1e-3),
                 metrics=["accuracy"])

Note que en el siguiente `fit()` definimos explícitamente el número de itaraciones que suceden en cada epoch: `steps_per_epoch`, y lo hacemos tanto para el conjunto de entrenmiento como para el de validación. En cualquier caso bebe calcularse simplemente por medio de la división entre el número de muestras y el tamaño del _batch_.

In [ ]:
# Entrenamos el modelo
history = conv_net.fit(train_generator,
                                 steps_per_epoch=2000//batch_size,
                                 epochs=10,
                                 validation_data=validation_generator,
                                 validation_steps=1000//batch_size,
                                 verbose=1)

Ahora veamos las pérdidas del modelo y la progresión del _accuracy_ a lo largo de las épocas :

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(121)
plt.plot(history.history["loss"], label="entrenamiento")
plt.plot(history.history["val_loss"], label="validación")
plt.title("Pérdida"); plt.xlabel("Época"); plt.legend()
plt.subplot(122)
plt.plot(history.history["accuracy"], label="entrenamiento")
plt.plot(history.history["val_accuracy"], label="validación")
plt.title("Accuracy"); plt.xlabel("Época"); plt.legend()

Podemos observar las representaciones intermedias o *feature maps* que fueron aprendidos por la red.
- Veamos un ejemplo de esto para  imágenes  del conjunto de entrenamiento:

> Puete tomar al rededor de 3 minutos en ejecutar con GPU

In [ ]:
dummy_data = np.random.rand(1, 150, 150, 3)  # Example with shape (1, 150, 150, 3)
conv_net.predict(dummy_data)

In [ ]:
# Definimos un modelo que mostrará la salida de cada una de las capas de
# la red convolucional original, excepto las últimas tres (densas).
successive_outputs = [layer.output for layer in conv_net.layers[:-3]]
visualization_model = tf.keras.models.Model(inputs=conv_net.layers[0].input, outputs=successive_outputs)

# Seleccionamos al azar y cargamos la imagen de un perro
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]
img_path = random.choice(dog_img_files)
img = np.array(tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150)))
img = np.expand_dims(img, axis=0)

# Aplicamos el preprocesamiento que usamos en el entrenamiento
img = img/255

# Obtenemos los feature maps
feature_maps = visualization_model.predict(img)
# Extraemos los nombres de cada capa
layer_names = [layer.name for layer in conv_net.layers[:-3]]

In [ ]:
# Mostramos la imagen original
plt.figure(figsize=(7,7))
plt.imshow(img[0])
plt.axis("off")
plt.title("Imagen original")

# Mostramos los feature maps:
for lab, fm in zip(layer_names, feature_maps):
    fig, ax = plt.subplots(6, 6, figsize=(15, 15))
    fig.suptitle(lab)
    for filter_i in range(fm.shape[-1]):
        ax[filter_i//6, filter_i%6].imshow(fm[0,:,:,filter_i])
        ax[filter_i//6, filter_i%6].axis("off")

Ahora aplicamos la red completa para obtener la predicción :

In [ ]:
prob = conv_net(img).numpy()[0,0]
print(f"Probabilidad de ser perro: {prob}\n\n")

La única salida de la red neuronal se interpreta como la probabilidad de ser perro, pues el generador de datos asignó a la clase perro la clase positiva (1), como se puede observar en el atributo `class_indices` del generador `train_generator`:

In [ ]:
print(train_generator.class_indices)

Realizamos el mismo proceso para una imagen de gato:

In [ ]:
# Seleccionamos al azar y cargamos la imagen de un gato
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
img_path = random.choice(cat_img_files)
img = np.array(tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150)))
img = np.expand_dims(img, axis=0)

# Aplicamos el preprocesamiento que usamos en el entrenamiento
img = img/255

# Obtenemos los feature maps
feature_maps = visualization_model.predict(img)
# Extraemos los nombres de cada capa
layer_names = [layer.name for layer in conv_net.layers[:-3]]

In [ ]:
# Mostramos la imagen original
plt.figure(figsize=(7,7))
plt.imshow(img[0])
plt.axis("off")
plt.title("Imagen original")

# Mostramos los feature maps:
for lab, fm in zip(layer_names, feature_maps):
    fig, ax = plt.subplots(6, 6, figsize=(15, 15))
    fig.suptitle(lab)
    for filter_i in range(fm.shape[-1]):
        ax[filter_i//6, filter_i%6].imshow(fm[0,:,:,filter_i])
        ax[filter_i//6, filter_i%6].axis("off")

Ahora aplicamos la red completa para obtener la predicción, en este caso usamos el complemento de la salida, pues la clase gato corresponde a la clase negativa (0):

In [ ]:
prob = 1.0 - conv_net(img).numpy()[0,0]
print(f"Probabilidad de ser gato: {prob}\n\n")

# **Recursos adicionales**
----
Los siguientes enlaces corresponden a sitios en donde encontrará información muy útil para profundizar las redes convolucionales :

* https://machinelearningmastery.com/how-to-visualize-filters-and-feature-maps-in-convolutional-neural-networks/
* https://en.wikipedia.org/wiki/Kernel_(image_processing)
* https://www.tensorflow.org/tutorials/images/classification
* https://medium.com/@Joocheol_Kim/take-a-look-inside-the-cnn-eb65d0dfdf94
* _Origen de los íconos_
    - Flaticon. Pixels free icon [PNG]. https://www.flaticon.com/free-icon/pixels_1881511?_gl
    - Blog. ConvNet diagram [PNG]. https://4.bp.blogspot.com/-uHgA5CQk22A/VT3_7lLLXtI/AAAAAAAAOC0/YMAyuywwaQ0/s1600/convnet.png
    - Wikipedia. Discrete 2D Convolution Animation [GIF]. https://en.m.wikipedia.org/wiki/Convolution#/media/File%3A2D_Convolution_Animation.gif
    - Nadeem Qazi. The window incorporates the depth, but it only moves along two dimensions of the image [GIF]. https://miro.medium.com/max/738/1*Q7NXeOlDkm4xlNrNQOS67g.gif
    - Bouvet. A Pooling example with a stride of 2 and a filter size of 2x2 [GIF]. https://www.bouvet.no/bouvet-deler/understanding-convolutional-neural-networks-part-1/_/attachment/inline/e60e56a6-8bcd-4b61-880d-7c621e2cb1d5:6595a68471ed37621734130ca2cb7997a1502a2b/Pooling.gif    
    - Kaggle.  What is Image Data Augmentation [JPG]. https://www.kaggle.com/getting-started/190280


   

# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
  * [Juan Sebastián Lara](https://http://juselara.com/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).


**Universidad Nacional de Colombia** - *Facultad de Ingeniería*